In [4]:
import os
import requests
import time
import pandas as pd
from tqdm.notebook import tqdm

In [5]:
url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
query = """
    PREFIX wikibase: <http://wikiba.se/ontology#>
    PREFIX wd: <http://www.wikidata.org/entity/>
    PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    SELECT ?company ?label
    WHERE {
        ?company wdt:P31/wdt:P279* wd:Q4830453;
                 rdfs:label ?label.
        FILTER(LANG(?label) = "en").
    }
    """

data = requests.get(url, params={'query': query, 'format': 'json'}).json()

In [6]:
companies = {}

for result in data["results"]["bindings"]:
    companies[result['company']['value'].split('/')[-1]] = result['label']['value']

In [10]:
!touch company_aliases.tsv

In [15]:
company_aliases = {}

def read_company_aliases():
    res = {}
    with open('company_aliases.tsv', 'r') as f:
        for line in f.readlines():
            items = line.strip().split('\t')
            if len(items) > 2:
                company = items[0]
                company_label = items[1]
                res[company] = (company_label, items[2:])
    return res

company_aliases = read_company_aliases()

def write_company_aliases(company_aliases):
    with open('company_aliases.tsv', 'w') as f:
        for company in company_aliases:
            company_label, aliases = company_aliases[company]
            f.write(company + '\t' + companies[company] + '\t' + '\t'.join(aliases) + '\n')

write_company_aliases(company_aliases)

In [ ]:
with open('company_aliases.tsv', 'w') as f:
    for n, company in tqdm(enumerate(companies), total=len(companies)):
        if company not in company_aliases:
            print(n, company, companies[company])
            query = """
                PREFIX wikibase: <http://wikiba.se/ontology#>
                PREFIX wd: <http://www.wikidata.org/entity/>
                PREFIX wdt: <http://www.wikidata.org/prop/direct/>
                PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

                SELECT ?altLabel 
                WHERE {
                    wd:$$company$$ skos:altLabel ?altLabel
                    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
                }
                """.replace("$$company$$", company)
            try:
                data = requests.get(url, params={'query': query, 'format': 'json'}).json()
                aliases = []
                for result in data["results"]["bindings"]:
                    aliases.append(result['altLabel']['value'])

                company_aliases[company] = (companies[company], aliases)
                f.write(company + '\t' + companies[company] + '\t' + '\t'.join(aliases) + '\n')
            except:
                print("except")
                time.sleep(2)

9 Q493500 The Namibian
10 Q13709 Le Canard enchaîné
12 Q20580 Seelter Trjoue
13 Q147261 Zapf Umzüge
14 Q169356 ZebeMusic
16 Q203704 Zimmermann
17 Q294609 AMA Verlag
20 Q468157 Edition Peters
23 Q569779 Universal Edition
24 Q262625 The Sowetan
26 Q269963 7Days
except
29 Q278151 Spiegel Online
30 Q279739 Business Line
31 Q283570 Moskauer Deutsche Zeitung
except
32 Q284947 The Christian Science Monitor
35 Q59705 Business Recorder
38 Q796316 BMG Rights Management
except
41 Q940752 Bourne Co. Music Publishers
42 Q1029842 Robert Lienau
43 Q569851 Shekulli
except
44 Q572893 Eleftheros Typos
47 Q575741 The Mercury News
49 Q582714 Panorama
except
50 Q35638 El Periódico de Catalunya
51 Q41288 The Daily
52 Q41369 Le Socialiste picard
except
53 Q41595 Hindustan Times
55 Q5193799 Cumbancha
56 Q5483919 Kaiku Entertainment
except
57 Q5497970 Frederick Harris Music
58 Q5505207 Frog Peak Music
59 Q5668906 Harry Fox Agency
except
62 Q6409789 Kimbel and Cabus
64 Q6859189 Miller Music Publishing Co.
65 Q6

In [ ]:
len(companies)